# Extra Credit

For upto an additional 15 extra credit points on the midterm exam, you can write another function that applies additional criteria on top of what the midterm already did.  To earn the addition points, you need write a function called `summarize_results(patients)` to take the output of your  `process_people()` function to apply a new business rule and summarize the results.

* Your function should take the dictionary created by `process_people()` as it's one input parameter: `patients`.
* If the PRIEST probably is less than 10%, change the `hospital` and `address` to both have a value of `'Home'` indicating patients with a lower probability of an adverse outcome can just go home rather than be admitted to a hospital.
* Summarize those results and count how many patients are now being sent to each location.  Your result should have the following format:
```
{
    "Home": 123,
    "Hospital A": 10,
    "Hospital B": 20,
    "Hospital C": 30
}
```

As always, your function needs good docstrings; but does not require any doctests in this case.

In [13]:
def priest(sex, age, respRate, oxySat, heartRate, sysBP, temp, alertness, inspOyg, perform):
    """
    (String, integer, integer, integer, integer, integer, float, String, String, String) -> float)

    Priest function calculates risk percentage like Predicts adverse outcome in adults presenting to ED with suspected COVID-19.

    Priest function expects sex, alertness, inspOxy and perform in String and remaining all are in float.

    Priest function calculates risk based on the algorithm provided in PRIEST mdcalc

    >>> priest("Male", 18, 12, 0.96, 115, 115, 38.1, "ALERT", "AIR", "unrestricted normal activity")
    0.03

    >>> priest("FEMALE", 18, 24, 0.96, 41, 115, 35.1, "confused or not alert", "AIR", "unrestricted normal activity")
    0.18

    >>> priest("FEMALE", 40, 8, 0.89, 105, 101, 38.1, "ALERT", "supplemental oxygen", "unrestricted normal activity")
    0.34

    """

    sex = sex.lower()
    score = 0

    if sex=="male":
        score += 1
    if age > 80:
        score = score + 4
    elif age >= 50 and age<=65:
        score = score + 2
    elif age >= 66 and age<=80:
        score = score + 3
    if respRate < 9:
        score = score +  3
    elif respRate >= 9 and respRate <= 11:
        score = score +  1
    elif respRate >= 21 and respRate <= 24:
        score = score +  2
    elif respRate > 24:
        score = score +  3
    if oxySat >= 0.94 and oxySat <= 0.95:
        score = score +  1
    elif oxySat >= 0.92 and oxySat <= 0.93:
        score = score +  2
    elif oxySat < 0.92:
        score = score +  3
    if heartRate < 41:
        score = score +  3
    elif heartRate >= 41 and heartRate <= 50:
        score = score +  1
    elif heartRate >= 91 and heartRate <= 110:
        score = score +  1
    elif heartRate >= 111 and heartRate <= 130:
        score = score +  2
    elif heartRate > 130:
        score = score +  3
    if sysBP < 91:
        score = score +  3
    elif sysBP >= 91 and sysBP <= 100:
        score = score +  2
    elif sysBP >= 101 and sysBP <= 110:
        score = score +  1
    elif sysBP > 219:
        score = score +  3
    if temp < 35.1:
        score = score +  3
    elif temp >= 35.1 and temp <= 36.0:
        score = score +  1
    elif temp >= 38.1 and temp <= 39.0:
        score = score +  1
    elif temp > 39.0:
        score = score +  2
    alertness = alertness.lower()
    if alertness == "confused or not alert":
        score = score +  3
    inspOyg = inspOyg.lower()
    if inspOyg == "supplemental oxygen":
        score = score +  2
    perform = perform.lower()
    if perform == "limited strenuous activity, can do light activity":
        score = score +  1
    elif perform == "limited activity, can self-care":
        score = score +  2
    elif perform == "limited self-care":
        score = score +  3
    elif perform == "bed/chair bound, no self-care":
        score = score +  4

    rPer = [0.01, 0.01, 0.02, 0.02, 0.03, 0.09, 0.15, 0.18, 0.22, 0.26, 0.29, 0.34, 0.38, 0.46, 0.47, 0.49, 0.55, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,0.99]
    if score>=26:
        return 0.99
    return rPer[score]

In [14]:
import json
import csv
import requests

def find_hospital(age, sex, rper):
    """
    (integer, String, float) -> String

    find_hospital function find the hospital based on age, sex and risk percentage.

    find_hospital function expects age, risk percentage in float and sex in String and produces hospital name in String

    find_hospital function find hospital on API by passing age, sex and rper as query parameters


    >>> find_hospital(40, "female", 0.34)
    'Emory Dunwoody Medical Center'

    >>> find_hospital(40, "female", 0.03)
    'Select Specialty Hospital - Northeast Atlanta'

    >>> find_hospital(18, "MALE", 0.26)
    'Southwest Hospital and Medical Center'

    """
    sex = sex.lower()
    httpurll = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"
    outPut = requests.get(httpurll, params={"age": age, "sex": sex, "risk_pct": rper})

    if outPut.status_code==200:
        outPutJSON = outPut.json()
        return outPutJSON['hospital']
    else:
        print("URL is not working and status code is : " + outPut.status_code)

In [15]:
import json
import csv
import requests

def get_address(hospName):
    """
    (String) -> String

    get_address function find the address of hospital based on hospital name from the url provided

    get_address function expects hospital name as String and return address in String

    get_address function find address by use API, where it has hospital name json data in it.

    >>> get_address("Select Specialty Hospital - Northeast Atlanta")
    '1821 CLIFTON ROAD NE'
    >>> get_address("Hughes Spalding Childrens Hospital")
    '1711 TULLIE CIRCLE NE'
    >>> get_address("Southwest Hospital and Medical Center")
    '501 FAIRBURN ROAD SW'
    """

    httpurll = "https://hds5210-data.s3.amazonaws.com/ga_hospitals.json"
    outPut = requests.get(httpurll)
    hospName=hospName.upper()
    if outPut.status_code==200:
        outPutJSON = outPut.json()
        if hospName in outPutJSON:
            return outPutJSON[hospName]["ADDRESS"]
        else:
            return " "
    else:
        print("URL is not working and status code is : " + outPut.status_code)

In [16]:
import json
import csv
import requests

def process_people(floc):
    """
    (String) -> Dictionary

    process_people function finds the risk percentage, hospital to visit and hospital address
    from group of patients.

    process_people function accepts floc as String and return Dictionary in json format

    It gathers data in the csv file from the API and computes risk percentage, hospital to visit
    and address of the hospital and append each patient with information including all factors and risk percentage,
    hospital name and address to the dictionary in json format.

    """

    appendJSON = {}
    outPut = requests.get(floc)
    if outPut.status_code==200:
       fileData = outPut.text.split("\n")
       fileDataWithoutDelimiter = csv.reader(fileData, delimiter='|')
       patient, sex, age, respRate, oxySat, heartRate, sysBP, temp, alertness, inspOyg, perform = next(fileDataWithoutDelimiter)
       for x in fileDataWithoutDelimiter:
           if not any(field != '' for field in x):
               break
           patient, sex, age, respRate, oxySat, heartRate, sysBP, temp, alertness, inspOyg, perform = x
           rPer = priest(sex, float(age), float(respRate), float(oxySat), float(heartRate), float(sysBP), float(temp), alertness, inspOyg, perform)
           hospName = find_hospital(int(age), sex, rPer)
           adds = get_address(hospName)
           df = [sex, float(age), float(respRate), float(oxySat), float(heartRate), float(sysBP), float(temp), alertness, inspOyg, perform, rPer, hospName, adds]
           appendJSON[patient] = df

       return appendJSON


    else:
        print("URL is not working and status code is : " + outPut.status_code)

In [17]:
import json
import csv
import requests

"""
It compares the actual data from API with the processed data
from above methods and displays whether both are equal or not
"""

httpurll = "https://hds5210-data.s3.amazonaws.com/people_results.json"

outPut = requests.get(httpurll)
actualData = outPut.json()
computedData = process_people("https://hds5210-data.s3.amazonaws.com/people.psv")
if json.loads(json.dumps(actualData)) == json.loads(json.dumps(computedData)):
    print("Actual JSON Data and Computed JSON Data are equal")
else:
    print("Both Are Not Equal")

Actual JSON Data and Computed JSON Data are equal


In [18]:
def summarize_results(patients):
    """

    (dictionary)->dictionary

    The summarize_results function is used to mark a
    patient as 'Home' when their risk probability is less than 10%.
    A dictionary with a clear hospital count is returned.


    The summarize function accepts any patient data as input
    arguments and outputs a single hospital count in dictionary format.

    """
    dt = set()
    hospitalJson = {}
    ind = 0
    for m, n in patients.items():
        if n[10] <= 0.1:
            patients[m][11]="Home"
            patients[m][12]="Home"
        dt.add(n[11])
    for m in dt:
        hospitalJson[m]=0
    my_list = list(dt)
    while ind < len(my_list):
        for m, n in patients.items():
            if my_list[ind] == n[11]:
                hospitalJson[my_list[ind]] = hospitalJson[my_list[ind]] + 1
        ind = ind +1
    return hospitalJson

summarize_results(process_people("https://hds5210-data.s3.amazonaws.com/people.psv") )

{'Southwest Hospital and Medical Center': 16,
 'Emory Dunwoody Medical Center': 5,
 'Select Specialty Hospital - Northeast Atlanta': 32,
 'Wesley Woods Geriatric Hospital': 20,
 'Hughes Spalding Childrens Hospital': 1,
 'Childrens Healthcare of Atlanta at Scottish Rite': 1,
 'Home': 24}